In [16]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.datasets import make_regression
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import median_absolute_error
from sklearn.metrics import explained_variance_score
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_absolute_error
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR
import codesModel as codeM


In [17]:
# Opciones de pandas
rows, cols = 50, 50
pd.set_option("display.max.columns", cols)
pd.set_option("display.max.rows", rows)
data = pd.read_csv('./dataBacktesting.csv', header=[0,1], index_col=0)
data.set_index(pd.DatetimeIndex(list(data.index)).strftime('%Y-%m-%d'), inplace=True)
data

Attributes       Close               CloseNext                    High  \
Symbol             IBM         PEP         IBM         PEP         IBM   
1985-03-01   14.030176    1.134058   13.939824    1.132171   14.120527   
1985-03-04   13.939824    1.132171   13.939824    1.155759   14.081804   
1985-03-05   13.939824    1.155759   13.720400    1.138069   14.030175   
1985-03-06   13.720400    1.138069   13.436437    1.135119   13.991452   
1985-03-07   13.436437    1.135119   13.384818    1.132171   13.772025   
...                ...         ...         ...         ...         ...   
2021-04-23  140.856171  145.830002  140.005692  143.360001  142.023140   
2021-04-26  140.005692  143.360001  140.440826  142.889999  142.042927   
2021-04-27  140.440826  142.889999  141.419891  141.830002  140.984752   
2021-04-28  141.419891  141.830002  142.646194  143.940002  141.815465   
2021-04-29  142.646194  143.940002         NaN         NaN  147.096471   

Attributes                     Low                    Open              \
Symbol             PEP         IBM         PEP         IBM         PEP   
1985-03-01    1.145750   13.836567    1.116522   13.888196    1.122367   
1985-03-04    1.143964   13.875288    1.126275   14.068897    1.138068   
1985-03-05    1.161655   13.901102    1.132172   13.978546    1.135120   
1985-03-06    1.158708   13.707493    1.135120   13.952730    1.155759   
1985-03-07    1.143964   13.423529    1.123326   13.668767    1.132171   
...                ...         ...         ...         ...         ...   
2021-04-23  146.339996  139.392529  145.059998  139.748551  145.509995   
2021-04-26  145.770004  139.916675  142.979996  140.846290  145.550003   
2021-04-27  143.179993  139.194754  142.130005  140.124370  142.619995   
2021-04-28  143.690002  140.529842  141.729996  141.340774  143.490005   
2021-04-29  144.429993  141.400107  141.970001  142.537409  142.100006   

Attributes   Volume                bop                 ema20              \
Symbol          IBM      PEP       IBM       PEP         IBM         PEP   
1985-03-01  5532000  3880800  0.500000  0.399990   14.030176    1.134058   
1985-03-04  3560400  3142800 -0.625000 -0.333343   14.021571    1.133879   
1985-03-05  4040000  5153400 -0.300000  0.700006   14.013786    1.135962   
1985-03-06  4120000  3722400 -0.818182 -0.749982   13.985844    1.136163   
1985-03-07  7028000  4624200 -0.666667  0.142847   13.933520    1.136064   
...             ...      ...       ...       ...         ...         ...   
2021-04-23  4556000  4354700  0.421050  0.250006  133.983197  143.003346   
2021-04-26  4710800  5817700 -0.395343 -0.784945  134.556768  143.037313   
2021-04-27  3884000  4693700  0.176791  0.257150  135.117154  143.023283   
2021-04-28  3768100  4962700  0.061540 -0.846938  135.717415  142.909637   
2021-04-29  4353900  4699900  0.019097  0.747969  136.377299  143.007767   

Attributes      ema200                   lower                    ma20  \
Symbol             IBM         PEP         IBM         PEP         IBM   
1985-03-01   14.030176    1.134058         NaN         NaN         NaN   
1985-03-04   14.029277    1.134040         NaN         NaN         NaN   
1985-03-05   14.028387    1.134256         NaN         NaN         NaN   
1985-03-06   14.025323    1.134294         NaN         NaN         NaN   
1985-03-07   14.019463    1.134302         NaN         NaN         NaN   
...                ...         ...         ...         ...         ...   
2021-04-23  122.718700  136.941129  127.509733  140.396457  134.125881   
2021-04-26  122.890710  137.004998  127.265145  140.455211  134.382513   
2021-04-27  123.065338  137.063556  127.071209  140.392265  134.686616   
2021-04-28  123.247970  137.110983  126.948194  140.293812  135.096041   
2021-04-29  123.440987  137.178933  126.987805  140.564317  135.638976   

Attributes                  macd            momentum              signal  \
Symbol             PEP       IBM       PEP    

In [18]:
#se borran estas fechas las cuales estan vacias en todas las acciones
date_drops = ['2021-04-29']
data.drop(date_drops, axis=0, errors='ignore', inplace=True)
data.set_index(pd.DatetimeIndex(list(data.index)).strftime('%Y-%m-%d'), inplace=True)

In [19]:
#borrar desde 1980
data.drop(data.index[:data.index.get_loc('1985-03-05')], axis=0, inplace=True)

In [20]:
# missing values in the data are interpolated using forward-fill (‘ffill ()’)method to propagate last valid observation forward.
data = data.fillna(method='bfill')

In [21]:
#se borran estos datos debido a la alta correlacion
attributes_to_erase = ('High','Low','Open','ma20','lower','upper','ema20')
for i in attributes_to_erase:
    data=data.drop(i, axis=1, level=0)

In [22]:
data.drop(data.index[data.index.get_loc('2021-04-01'):], axis=0, inplace=True)

In [23]:
data

Attributes       Close               CloseNext               Volume           \
Symbol             IBM         PEP         IBM         PEP      IBM      PEP   
1985-03-05   13.939824    1.155759   13.720400    1.138069  4040000  5153400   
1985-03-06   13.720400    1.138069   13.436437    1.135119  4120000  3722400   
1985-03-07   13.436437    1.135119   13.384818    1.132171  7028000  4624200   
1985-03-08   13.384818    1.132171   13.578421    1.135119  6892000  2764800   
1985-03-11   13.578421    1.135119   13.617147    1.149862  4204000  1609200   
...                ...         ...         ...         ...      ...      ...   
2021-03-25  131.599625  139.630005  134.873047  142.699997  5554000  5827600   
2021-03-26  134.873047  142.699997  134.358780  144.809998  5567600  6785600   
2021-03-29  134.358780  144.809998  133.231384  142.539993  4622700  6105800   
2021-03-30  133.231384  142.539993  131.787506  141.449997  4790400  4372700   
2021-03-31  131.787506  141.449997  131.757843  141.279999  4945100  6257900   

Attributes       bop                ema200                  macd            \
Symbol           IBM       PEP         IBM         PEP       IBM       PEP   
1985-03-05 -0.300000  0.700006   14.028387    1.134256 -0.012772  0.001615   
1985-03-06 -0.818182 -0.749982   14.025323    1.134294 -0.034491  0.001568   
1985-03-07 -0.666667  0.142847   14.019463    1.134302 -0.073766  0.001279   
1985-03-08 -0.466667 -0.400007   14.013148    1.134281 -0.107814  0.000802   
1985-03-11  0.812500 -0.333371   14.008822    1.134289 -0.117817  0.000655   
...              ...       ...         ...         ...       ...       ...   
2021-03-25  0.789627  0.235608  120.173539  135.452192  2.532394  1.117782   
2021-03-26  0.919646  0.795867  120.319802  135.524310  2.919567  1.577161   
2021-03-29 -0.076919  0.763778  120.459494  135.616705  3.148612  2.087419   
2021-03-30 -0.506666 -0.433459  120.586577  135.685593  3.202247  2.282323   
2021-03-31 -0.639999 -0.865744  120.698029  135.742950  3.092595  2.322064   

Attributes  momentum              signal              std20d            
Symbol           IBM       PEP       IBM       PEP       IBM       PEP  
1985-03-05  0.000000  0.023587 -0.003708  0.000299  0.324364  0.047906  
1985-03-06 -0.219424 -0.017689 -0.009864  0.000553  0.324364  0.047906  
1985-03-07 -0.283963 -0.002950 -0.022645  0.000698  0.324364  0.047906  
1985-03-08 -0.051619 -0.002948 -0.039678  0.000719  0.324364  0.047906  
1985-03-11  0.193603  0.002948 -0.055306  0.000706  0.324364  0.047906  
...              ...       ...       ...       ...       ...       ...  
2021-03-25  2.422943  0.820007  2.005483  0.046248  4.155823  3.492027  
2021-03-26  3.273422  3.069992  2.188299  0.352431  4.345533  3.866542  
2021-03-29 -0.514267  2.110001  2.380362  0.699429  4.474108  4.412754  
2021-03-30 -1.127396 -2.270004  2.544739  1.016007  4.342329  4.590686  
2021-03-31 -1.443878 -1.089996  2.654310  1.277219  4.193578  4.445989  

[9093 rows x 18 columns]

In [24]:
Params2 = {'max_features': 0.7, 'max_depth': 10, 'min_samples_leaf': 2, 'n_estimators': 1000, 'oob_score': True, 'criterion':"mae"}
paramsSVR = {
    "kernel": 'linear',
    "C":5.0,
    "epsilon": 0.001,
    
}
ParamsMLP = {
    "hidden_layer_sizes": (200,200,200),
    "activation": 'logistic',
    "solver":'sgd',
    "alpha":0.001,
    "learning_rate":'adaptive'
}
modelRF = RandomForestRegressor(max_features= Params2['max_features'], max_depth= Params2['max_depth'],  
                              min_samples_leaf= Params2['min_samples_leaf'], n_estimators=Params2['n_estimators'], 
                              oob_score= Params2['oob_score'], criterion= Params2['criterion'])
modelMLP=MLPRegressor(max_iter=20000, hidden_layer_sizes=ParamsMLP['hidden_layer_sizes'], activation=ParamsMLP['activation'],solver=ParamsMLP['solver'])
modelSVR = SVR(kernel=paramsSVR['kernel'], C=paramsSVR['C'], epsilon=paramsSVR['epsilon'])

In [25]:
data_train_scaler = data.drop(data.index[data.index.get_loc('2020-09-30'):], axis=0)
data_test_scaler = data.drop(data.index[:data.index.get_loc('2020-09-30')], axis=0)

In [26]:
rfResult=codeM.allModel(data_train_scaler,data_test_scaler,modelRF,8, isScaled=True)

'PEP'

'IBM'

In [27]:
mlpResult=codeM.allModel(data_train_scaler,data_test_scaler,modelMLP,8, isSacled=True)
svrResult=codeM.allModel(data_train_scaler,data_test_scaler,modelSVR,8, isScaled=True)

TypeError: allModel() got an unexpected keyword argument 'isSacled'

In [ ]:
data.index[data.index.get_loc('2020-09-30'):]

In [ ]:
def multi_graph(stock):
    arr_len = len(svrResult[stock][0])
    svr_arr = (svrResult[stock][0],np.reshape(svrResult[stock][1], (arr_len)))
    rf_arr = (rfResult[stock][0],np.reshape(rfResult[stock][1], (arr_len)))
    mlp_arr = (mlpResult[stock][0],np.reshape(mlpResult[stock][1], (arr_len)))
    
    fig1,ax1 = plt.subplots(figsize=(20,10))

    plt.plot(data.index[data.index.get_loc('2020-09-30'):], mlp_arr[0], color='#4A5ECF', label='MLP')
    plt.plot(data.index[data.index.get_loc('2020-09-30'):], svr_arr[0], color='#7D6F1E', label='SVR')
    plt.plot(data.index[data.index.get_loc('2020-09-30'):], rf_arr[0], color='#05B543', label='RF')
    plt.plot(data.index[data.index.get_loc('2020-09-30'):], svr_arr[1], color='#992E74', label='Actual Price')


    plt.grid(which='major', color='#cccccc', alpha=0.5)

    plt.legend(shadow=True)
    plt.title('Model Prediction Comparison for {} stock'.format(stock), family='DejaVu Sans', fontsize=12)
    plt.xlabel('Timeline', family='DejaVu Sans', fontsize=10)
    plt.ylabel('Stock Price Value', family='DejaVu Sans', fontsize=10)
    plt.xticks(rotation=45, fontsize=11)
    plt.show()

In [ ]:
for stock in ['PEP','IBM']:
    multi_graph(stock)